# Step 1: Obtaining our Data

In [ ]:
# Load the libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
from statsmodels.formula.api import ols

### import and read file:

In [ ]:
kc = pd.read_csv('kc_house_data.csv', 'r', delimiter=',')

#Checking that the import was successful.
kc.head()

** Take another quick glance to make sure everything is imported properly**

In [ ]:
kc.tail()

In [ ]:
kc.sample(20)

In [ ]:
print(kc.columns)

Import the Column Names and Descriptions that were included as a separate file.

In [ ]:
from IPython.display import display, Markdown

with open('column_names.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

Wanting further clarification at this point I went onto slack and I asked about the living15 and lot15 columns as well as price.
Price = The price the house sold for
living15 and lot15 is the average of the 15 nearest neighbors. 

### Found a useful glossary of terms when dealing with Kings County Residential Properties<br>
##### I'm not going to waste time learning to scrape a website right now, so I will include a link and fix this if I have extra time at the end of the project
https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#g

# Step 2: Scrubbing

#### Subsampling to reduce Filesize

In [ ]:
kc.info()

In [ ]:
kc.shape

21,597 Rows & 21 Columns at 3.5MB is not a huge dataset.  I'm going to try to skip subsampling at this time and hope that run times are not too long.

#### Correcting Datatypes

In [ ]:
kc.info()

In [ ]:
kc.dtypes

**Take note of the columns we will need to explore further because they are more than likely the wrong data type.**
* 1. Date is object should probably be datetime.
* 2. Waterfront should be Boolean
* 3. I'm not sure about view, I was expecting object
* 4. yr_renovated and floors should be whole numbers but float64 will get the job done, not worth messing with.

In [ ]:
kc['date'].dtype

I  know that I need to change the object type to datetime.  Remember that Date is the date the house was sold.  I'm also curious about the range of dates, so I will check that while I am cleaning.

In [ ]:
kc['date'] = pd.to_datetime(kc['date'])
kc['date'].head()

Great! The data type has been successfully changed to datetime in the format of yyyy-mm-dd.

In [ ]:
# earliest date recorded
kc['date'].min()

In [ ]:
# latest date recorded
kc['date'].max()

In [ ]:
# range of dates
kc['date'].max() - kc['date'].min()

In [ ]:
kc.yr_built.head()

yr_built should be categorical.

In [ ]:
kc.yr_built= kc.yr_built.astype("category")

In [ ]:
kc.yr_renovated.head()

Same as with yr_built, probably... However, I will need to deal with those NaN's in a bit.

In [ ]:
kc.dtypes

I missed sqft_basement, currently stored as an object.

In [ ]:
kc['sqft_basement'].dtype

In [ ]:
kc['sqft_basement'].sample(5)

In [ ]:
kc.loc[kc['sqft_basement'] == '?',['sqft_living','sqft_above']]

if the sqft_above is more than sqft_living I can be pretty sure that they recorded the basements square footage as part of the homes square footage(which you are not supposed to do) So I can replace the ? with the difference.  

In [ ]:
kc['sqft_basement'].value_counts()

In [ ]:
kc.loc[(kc["sqft_basement"] == '?') & (kc["sqft_above"]!=kc['sqft_living'])]

So, I may be able to figure out 170 basement sizes

In [ ]:
print(454-170)

So, there are 284 rows where the sqft_above match the sqft_living and there is a ? mark as a placeholder for basement.  There is no way to find out the square footage of these basements if they even have a basement.  So, there value will now be 0.  And by subtracting the sqft_living from the sqft_above I am able to find the square footage of 170 homes which did not previously have their footage listed.

In [ ]:
kc['sqft_basement'].describe()

12826 homes do not have a basement out of 21,597 homes total.  

In [ ]:
#Replace ? with difference between sqft_above and sqft_living

kc['sqft_basement'].replace(to_replace='?', value = kc['sqft_above']-kc['sqft_living'], inplace=True)
kc.describe()

In [ ]:
kc['sqft_basement'].describe()

Waterfront needs to be fixed as well.

In [ ]:
kc['waterfront'].sample(10)

In [ ]:
kc['waterfront'].unique()

In [ ]:
kc['waterfront'].isnull().sum()

In [ ]:
kc['waterfront'].value_counts()

In [ ]:
# most properties are not waterfront, these are 
#realestate listings so waterfront would be listed more than likely
#nan is probably no
kc['waterfront'].fillna(0, inplace=True)

In [ ]:
kc['waterfront'].isnull().sum()

In [ ]:
# Change datatype to Boolean 0 is no, 1 is yes.
kc['waterfront'] = kc['waterfront'].astype('bool')

In [ ]:
kc['waterfront'].dtype

In [ ]:
#split the data frame on Boolean values
#create a waterfront database
waterfront = kc[kc['waterfront'] == True]

In [ ]:
waterfront['waterfront'].describe()

In [ ]:
no_water = kc[kc['waterfront'] == False]

In [ ]:
no_water['waterfront'].value_counts()

21,451 falls values to 146 is likely to mess up my regression. And I have already saved both in separate dataframes if I'd like to go back and utilize them, so I am dropping this column from my main DataFrame.


In [ ]:
kc = kc.drop(["waterfront"], axis=1)


In [ ]:
kc.info()

Detecting and dealing with Null Values and Placeholders

In [ ]:
# For each column, are there any NaN values?
kc.isnull().any()

View, and yr_renovated both have NaN values.    Also, there may be more "place holders" I need to deal with.  

In [ ]:
# for each column, how many rows are NaN?
kc.isnull().sum()

Clean up the View Column

In [ ]:
kc['view'].dtypes

In [ ]:
kc['view'].unique()

In [ ]:
kc['view'].isnull().sum()

In [ ]:
kc['view'].value_counts()

In [ ]:
# there are only 63 null views there are already
# over 19,000 views = 0

kc['view'].fillna(0, inplace=True)

In [ ]:
kc['view'].isnull().sum()

In [ ]:
# get z-score and assign it to variable for later use
z_views = np.abs(stats.zscore(kc['view']))
print(z_views)

In [ ]:
plt.hist(kc['view'])
print('Skewness = ', skew(kc['view']))
print('Kurtosis = ', kurtosis(kc['view']))

This column is heavily skewed and unlikely to add much value to my regression model due to the vast majority of properties having a view==0.  Also, I am not entirely certain the data is valid, because my best guess is that the view is how many times somebody viewed the property before buying it.  And, 0 seems unlikely.  Sure some people buy sight unseen, but not the majority like this graph would suggest.

In [ ]:
kc = kc.drop(["view"], axis=1)

In [ ]:
kc.info()

**yr_renovated**- should this be datetime?  No, it's just 1 year non-continuous.  It is however ordinal and needs to be classified as such. The First issue will be to deal with NaN values.

In [ ]:
kc['yr_renovated'].isnull().sum()

In [ ]:
kc['yr_renovated'].unique()

In [ ]:
kc['yr_renovated'].value_counts().head()

In [ ]:
# 3,842 houses have no data showing renovation with NaN
# 17,011 houses have no data showing renovation with value == 0
# combine these 2 for now
kc['yr_renovated'].fillna(0,inplace=True)

In [ ]:
# put renovated houses into their own seperate dataframe
renovated = kc[kc['yr_renovated'] == True]

In [ ]:
#there are too few renovated apartments, if I treat them as a category the high level of 0's is likely to mess up
# my analysis.  
kc = kc.drop(["yr_renovated"], axis=1)

In [ ]:
kc.info()

### Check for Multicollinearity 

In [ ]:
# I just took out vmin=-1, not sure if I should put back or not
correlation = kc.corr()
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(correlation, annot=True,vmin=-1, linewidths=0, cmap="RdBu_r")

In [ ]:
sns.set(style="white")

# Compute the correlation matrix
corr = kc.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(18, 14))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True,annot=True, linewidths=.5, cbar_kws={"shrink": .5});

### Delete a few columns that I know I won't be using.  


I need to remove some columns. The threshold can be as low as 0.5 after normalization, I know I ran the above check a little prematurely.  However, it gives me more confidence in preforming some edits I thought I would need to make.  

view seems has higher correlation with price than I would have expected.

**id** A notation for a house.


In [ ]:
# the id, should be unique for each house.
kc.id.duplicated().sum()

177 ID's appear more than once.  I want to pull these for further analysis.

In [ ]:
duplicated_id = pd.concat(g for _, g in kc.groupby("id") if len(g) > 1)
# repeated house ID's
duplicated_id

353 rows are placed into a separate dataframe because this is worth exploring further, possibly one of my three required questions.  These houses were all bought and sold again within a span of 390 days, a quick look over the other values shows the price going up almost every time.  These houses were bought and then sold for a profit.  Therefore the earlier date is the number which I should drop.  People buy houses that are under market value, fix them up a bit then sell them at or slightly above market value.  Meaning that the second price is a more accurate representation when determining market value based upon other variables.


In [ ]:
kc['id'].drop_duplicates(keep='last',inplace=True)

In [ ]:
kc.id.duplicated().sum()

In [ ]:
kc = kc.drop(["id"], axis=1)

Well, I tried my best to plot the geographic locations of some of these houses.  But, after 45 minutes of wasting time with geopandas I've decided it's not worth wasting time on for this project so I am dropping those columns as well.

In [ ]:
kc = kc.drop(["lat","long"], axis=1)

sqft_living and sqft_above should be the same because only above ground is supposed to be listed as the houses square footage.  It would seem that not everybody follows this rule since the columns are not actually identical.  However; since the columns are not identical it is easy to deduce that some people added their basement's square footage to square foot living making this column invalid.

In [ ]:
kc.info()

**a closer look at sqft_living**

In [ ]:
kc['sqft_living'].sample(10)

In [ ]:
#smallest house 370 sqft
# Largest house is 13,540 square feet
kc['sqft_living'].describe()

In [ ]:
# median sqft is 1,910
kc['sqft_living'].median()

In [ ]:
kc['sqft_living'].plot(kind='box');

In [ ]:
kc['sqft_living'].var()

**a closer look at sqft_above**

In [ ]:
np.round(kc['sqft_above'].describe(),2)

In [ ]:
kc['sqft_above'].plot(kind='box');

The sqft_living has a lot of higher values, confirming my earlier suspicion that some people were adding their basements to their sqft_living. 

In [ ]:
kc = kc.drop(["sqft_living"], axis=1)

### Identify and deal with categorical variables
I'm going to plot some of the columns I think are likely to be categorical.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(16,3))

for xcol, ax in zip(['bedrooms', 'bathrooms', 'floors','condition','grade',], axes):
    kc.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.5, color='r')

Well, bedrooms and bathrooms seem to not be categorical based on this plot...I'm not sure how it is continuous

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,3))

for xcol, ax in zip(['bedrooms', 'bathrooms'], axes):
    kc.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.5, color='r')

Well, they were just too close together.  What is a quarter bathroom?  I know that a half-bath has no shower.

From Trulia.com<br>
"Simply stated, a bathroom can consist of a sink, toilet, shower and bathtub. Each of these is considered 1/4. Generally a half bath, also known as a powder room, consists of a sink and a toilet. Usually a 3/4 bath has a sink, toilet and shower. And a full bath or 4/4 bath has a sink, toilet, shower and tub."

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(16,10))

for xcol, ax in zip(['zipcode','condition',], axes):
    kc.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.5, color='b')

In [ ]:
sns.scatterplot(x=kc['yr_built'],y=kc['price'], x_bins=kc['yr_built']);

only yr_built 
'bedrooms', 'bathrooms', 'floors','condition','grade'

In [ ]:
kc["bathrooms"].describe()

#### Normalize the data

**2. date**- Date house was sold <br>

I'll just double check the range and see how the dates are spread out.

In [ ]:
# range of dates
kc['date'].max() - kc['date'].min()

In [ ]:
#import datetime as dt

In [ ]:
#kc['day_of_week'] = kc['date'].dt.dayofweek


In [ ]:
#quickly double checking that the days are correct.  Monday= 0 Sunday =6
#kc.head()

October 13th, 2014 is showing 0 which is Monday and that is correct according to my Calendar. So I am moving on to seasons.  Spring runs from March 1 to May 31;Summer runs from June 1 to August 31;Fall (autumn) runs from September 1 to November 30; Winter runs from December 1 to February 28. 

**3. Price**
Normalize price, I should remove all the other outliers in different columns before messing too much with price.

In [ ]:
kc['price'].mean()

In [ ]:
kc['price'].mode()

In [ ]:
np.round(kc['price'].describe())

In [ ]:
# Before removing outliers
sns.boxplot(x=kc['price'])

In [ ]:
kc['price']= np.log10(kc['price'])
#using log 10 since currency is a base 10 and I want to normalize the data a bit

In [ ]:
sns.boxplot(x=kc['price'])

In [ ]:
Q1_price = kc['price'].quantile(0.25)
Q3_price = kc['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
print(IQR_price)

In [ ]:
np.round(kc.price.describe())

In [ ]:
#remove the outliers of some of the other columns
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]


In [ ]:
kc['bedrooms'].min()

In [ ]:
kc['bedrooms'].max()

In [ ]:
kc['bedrooms'].value_counts()

In [ ]:
sns.boxplot(kc['bedrooms'])

In [ ]:
# eliminate rows with outlier categorical data based upon IQR
kc= kc[(kc['bedrooms'] <= 5) & (kc['bedrooms'] >=2)]

In [ ]:
kc.shape

In [ ]:
sns.boxplot(kc['bedrooms'])

I'm eliminating outliers with Bedrooms, hopefully this will take care of some other categories with extreme outliers.

In [ ]:
sns.boxplot(kc['floors']);

In [ ]:
sns.boxplot(kc['bathrooms']);

In [ ]:
kc['bathrooms'].value_counts()

In [ ]:
kc['bathrooms'].quantile(0.25)

In [ ]:
#kc= kc[(kc['bathrooms'] <= 0.75) & (kc['bathrooms'] >=3.5)]

In [ ]:
kc.head(5)

# Oh Shit! I fell asleep and forgot where I was at!

I got a little lost in what all I was doing above and don't want to spend the time figuring out how to fix it so some of the stuff below this may be repeats of above.  But I completely forgot to deal with categorical data, then tried to add it into the appropriate place above, didn't finish.  Went to sleep, lost my train of thought.  So.  Reboot!

In [ ]:
kc.head(5)

In [ ]:
kc.describe()

In [ ]:
kc.shape

In [ ]:
kc.info()

Looks like null values have been dealt with, and several columns have already been deleted.  I hope I don't have to go back and rework that part, I'm going forward and if I have time I'll go back and make a nice clean notebook.  But, this is serving as a lab notebook right now and the number one rule is to not erase any work. I guess I am being a bit like Virginia Wolfe or Ulysses and just writing in stream of consciousness for my first pass through this project.

In [ ]:
kc['bedrooms'].value_counts(dropna=False)

In [ ]:
kc.boxplot(column='price', by='bedrooms');
plt.show()

In [ ]:
kc['bathrooms'].value_counts(dropna=False).head()

In [ ]:
kc.boxplot(column='price', by='bathrooms');
plt.show()

In [ ]:
kc['floors'].value_counts(dropna=False)

In [ ]:
kc.boxplot(column='price', by='floors');
plt.show()

In [ ]:
kc['condition'].value_counts(dropna=False)

In [ ]:
kc.boxplot(column='price', by='condition');
plt.show()

In [ ]:
kc['grade'].value_counts(dropna=False)

In [ ]:
kc.boxplot(column='price', by='bedrooms');
plt.show()

In [ ]:
kc.boxplot(figsize=(20,20),column='price', by='yr_built', rot=85);
plt.show()

In [ ]:
kc.head()

In [ ]:
kc['bedrooms']= kc['bedrooms'].astype('category')
kc['bathrooms']= kc['bathrooms'].astype('category')
kc['floors']= kc['floors'].astype('category')
kc['condition']= kc['condition'].astype('category')
kc['grade']= kc['grade'].astype('category')
kc['yr_built']= kc['yr_built'].astype('category')
kc['zipcode']= kc['zipcode'].astype('category')
kc['day_of_week']= kc['day_of_week'].astype('category')

In [ ]:
kc.info()

In [ ]:
import re
pattern = re.compile('\$|d*|.|d{2}')
result = pattern.match('$17.89')
bool(result)

In [ ]:
# an assert method to make sure there are no missing values
assert pd.notnull(kc).all().all()

In [ ]:
# make sure there are no negative numbers
assert (kc >= 0).all().all()

In [ ]:
n_price = len(kc_no_outliers['price'])
n_price_bins = np.sqrt(n_price)
n_price_bins = int(n_price_bins)
_ = plt.hist(kc_no_outliers['price'], bins=n_price_bins)
_ = plt.xlabel('price')
_ = plt.ylabel('count')
plt.show()

Still skewed to the left a lot, but that's okay for now.

In [ ]:
kc2 = kc_no_outliers
kc2.head()

In [ ]:
bath_dummies = pd.get_dummies(kc2["bathrooms"], prefix="bath")

In [ ]:
bedroom_dummies = pd.get_dummies(kc2["bedrooms"], prefix="bed")

In [ ]:
floor_dummies= pd.get_dummies(kc2["floors"], prefix="flr")

In [ ]:
condition_dummies = pd.get_dummies(kc2["condition"], prefix="con")

In [ ]:
grade_dummies= pd.get_dummies(kc2["grade"], prefix="grd")

In [ ]:
yr_built_dummies= pd.get_dummies(kc2['yr_built'], prefix="yr")

In [ ]:
kc2 = kc2.drop(["bathrooms","bedrooms","floors",'condition','grade', 'yr_built', 'day_of_week'], axis=1)

In [ ]:
kc2 = pd.concat([kc2, bath_dummies, bedroom_dummies, floor_dummies, condition_dummies, grade_dummies], axis=1)

In [ ]:
kc2.head()

In [ ]:
print(list(kc2.columns))

In [ ]:
kc3 = kc_no_outliers

In [ ]:
# check for multicollinearity of features again
correlation = kc3.corr()
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(correlation, annot=True,vmin=-1, linewidths=0, cmap="RdBu_r")

sqft_above & sqft_living15 = 0.7
sqft_above & grade = 0.71
sqft_lot & sqft_lot15 = 0.72

In [ ]:
kc3 = kc3.drop(["sqft_lot15","sqft_living15","day_of_week"], axis=1)

In [ ]:
kc3.head()

In [ ]:
correlation = kc3.corr()
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(correlation, annot=True, vmin=-1, linewidths=1, cmap="RdBu_r")

In [ ]:
kc2 = kc2.drop(["sqft_lot15","sqft_living15"], axis=1)

In [ ]:
kc2.info()